# Frequency Cubes 2 Time Series
A script to facilitate generating a time series from processed videos.

Joe Ammatelli | jamma@uw.edu | 8-16-22

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import video_pixel_timeseries_analysis as vptsa

## Define regions to process, regional frequency statistics to record, and masking approach

In [31]:
REGION_CONFIG = {'drought' : {'roi' : [0, None, 0, None],
                              'sources' : ['masked', 'unmasked'],
                              'stats' : ['mean', 'median', 'mode']},
                }

MASK_APPROACH = 'median of max' 

## Define paths to video-datetime mapping, processed videos

In [32]:
TIMESERIES_PATH = './treesway_timeseries.csv'
PROCESSED_VIDS_PATH = 'processed_arrays'

## Load time series dataframe

In [33]:
# load dataframe
treesway_df = pd.read_csv(TIMESERIES_PATH, index_col=0, parse_dates=[0], infer_datetime_format=True)
treesway_df

,vid_name
datetime,
2022-05-15 11:30:00,GOPR0001
2022-05-15 12:00:00,GOPR0002
2022-06-07 09:00:00,GOPR0003


## Add columns for each metric specified in `REGION_CONFIG`

In [34]:
# add new column for each roi and for each stat ()
for region in REGION_CONFIG.keys():
    for source in REGION_CONFIG[region]['sources']:
        for stat in REGION_CONFIG[region]['stats']:
            treesway_df[f'{region}_{source}_{stat}'] = np.nan
treesway_df

,vid_name,drought_masked_mean,drought_masked_median,drought_masked_mode,drought_unmasked_mean,drought_unmasked_median,drought_unmasked_mode
datetime,,,,,,,
2022-05-15 11:30:00,GOPR0001,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-15 12:00:00,GOPR0002,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-07 09:00:00,GOPR0003,NaN,NaN,NaN,NaN,NaN,NaN


## For each video, aggregate frequency info for each region, source, and statistic

In [35]:
# for each video
for date, video in zip(treesway_df.index, treesway_df['vid_name']):
    freq = np.load(os.path.join(PROCESSED_VIDS_PATH, f'{video}_freq.npy'))
    pxx = np.load(os.path.join(PROCESSED_VIDS_PATH, f'{video}_pxx.npy'))
    
    for region in REGION_CONFIG.keys():
        # clip freq array to roi
        ymin, ymax, xmin, xmax = REGION_CONFIG[region]['roi']
        pxx_roi = pxx[:, ymin:ymax, xmin:xmax]
        
        # Get spectrum peaks and apply filter to remove noise
        peak_freq = vptsa.get_peak_freqs(freq, pxx)
        peak_freq_masked = vptsa.mask_peak_freq(MASK_APPROACH, peak_freq, pxx)
                  
        stats_df = vptsa.get_freq_specs(peak_freq, peak_freq_masked)
                  
        for source in REGION_CONFIG[region]['sources']:
            for stat in REGION_CONFIG[region]['stats']:
                treesway_df.loc[date, f'{region}_{source}_{stat}'] = stats_df.loc[source][stat]

In [36]:
treesway_df

,vid_name,drought_masked_mean,drought_masked_median,drought_masked_mode,drought_unmasked_mean,drought_unmasked_median,drought_unmasked_mode
datetime,,,,,,,
2022-05-15 11:30:00,GOPR0001,0.705923,0.696569,0.597059,0.672223,0.649741,0.597059
2022-05-15 12:00:00,GOPR0002,0.733821,0.755104,0.813639,0.640418,0.684862,0.813639
2022-06-07 09:00:00,GOPR0003,0.709996,0.714130,0.813639,0.682946,0.679008,0.813639


## Save table

In [37]:
treesway_df.to_csv(TIMESERIES_PATH)

## Plot variables of interest